# 데이터 불러오기

In [1]:
import os
import pandas as pd
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


# 데이터 전처리

In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [4]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head(11)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [6]:
movies.tail(10)

,movie_id,title,genre
3873,3943,Bamboozled (2000),Comedy
3874,3944,Bootmen (2000),Comedy|Drama
3875,3945,Digimon: The Movie (2000),Adventure|Animation|Children's
3876,3946,Get Carter (2000),Action|Drama|Thriller
3877,3947,Get Carter (1971),Thriller
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller


In [7]:
# 영화를 title로 입력할 것이기때문에 title이란 컬럼을 불러와야합니다.
ratings = pd.merge(ratings, movies[['title', 'movie_id']], on='movie_id', how='left')
ratings

,user_id,movie_id,counts,timestamp,title
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,978302109,James and the Giant Peach (1996)
2,1,914,3,978301968,My Fair Lady (1964)
3,1,3408,4,978300275,Erin Brockovich (2000)
4,1,2355,5,978824291,"Bug's Life, A (1998)"
...,...,...,...,...,...
836473,6040,1090,3,956715518,Platoon (1986)
836474,6040,1094,5,956704887,"Crying Game, The (1992)"
836475,6040,562,5,956704746,Welcome to the Dollhouse (1995)
836476,6040,1096,4,956715648,Sophie's Choice (1982)


In [8]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [9]:
# rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [10]:
#가장 인기 있는 영화 30개(인기순) 영화 id별 별점(시청횟수)많은 순
movie_count = ratings.groupby('movie_id')['counts'].count()
movie_count.sort_values(ascending=False).head(10)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
Name: counts, dtype: int64

In [11]:
# 내가 선호하는 영화를 5가지 골라서 rating에 추가
my_favorite = ['minority report', 'mission impossible', 'get out' , 'inception', 'gravity']

my_list = pd.DataFrame({'user_id': ['mino']*5, 'title': my_favorite, 'counts':[5]*5})

if not ratings.isin({'user_id':['mino']})['user_id'].any(): 
    ratings = ratings.append(my_list)                         

ratings.tail(10)         

,user_id,movie_id,counts,timestamp,title
836473,6040,1090.0,3,956715518.0,Platoon (1986)
836474,6040,1094.0,5,956704887.0,"Crying Game, The (1992)"
836475,6040,562.0,5,956704746.0,Welcome to the Dollhouse (1995)
836476,6040,1096.0,4,956715648.0,Sophie's Choice (1982)
836477,6040,1097.0,4,956715569.0,E.T. the Extra-Terrestrial (1982)
0,mino,NaN,5,NaN,minority report
1,mino,NaN,5,NaN,mission impossible
2,mino,NaN,5,NaN,get out
3,mino,NaN,5,NaN,inception
4,mino,NaN,5,NaN,gravity


In [14]:
# 인덱싱
userid_unique = ratings['user_id'].unique()
#movie_unique = ratings['movie_id'].unique()

movie_unique = ratings['title'].unique()

# user, movie indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(userid_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [16]:
# 인덱싱이 잘 되었는지 확인
print(user_to_idx['mino'])    
print(movie_to_idx['inception'])

6039
3631


In [17]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie_id column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')

ratings

user_id column indexing OK!!
movie_id column indexing OK!!


,user_id,movie_id,counts,timestamp,title
0,0,0,5,978300760.0,One Flew Over the Cuckoo's Nest (1975)
1,0,1,3,978302109.0,James and the Giant Peach (1996)
2,0,2,3,978301968.0,My Fair Lady (1964)
3,0,3,4,978300275.0,Erin Brockovich (2000)
4,0,4,5,978824291.0,"Bug's Life, A (1998)"
...,...,...,...,...,...
0,6039,3628,5,NaN,minority report
1,6039,3629,5,NaN,mission impossible
2,6039,3630,5,NaN,get out
3,6039,3631,5,NaN,inception


In [18]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

In [19]:
# csr_matrix
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3633 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [20]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

# Implicit AlternatingLeastSquares 모델의 선언
# 1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
# 2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 
# 3. use_gpu : GPU를 사용할 것인지 
# 4. iterations : epochs와 같은 의미입니다.
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [21]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3633x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [22]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [23]:
meissamatar, inception = user_to_idx['mino'], movie_to_idx['inception']
meissamatar_vector, inception_vector = als_model.user_factors[meissamatar], als_model.item_factors[inception]

In [24]:
# 내적하는 코드
np.dot(meissamatar_vector, inception_vector)

0.00033434454

In [25]:
getout = movie_to_idx['get out']
getout_vector = als_model.item_factors[getout]
np.dot(meissamatar_vector, getout_vector)

0.00017444883

In [26]:
# (id, 유사도 )
favorite_movie = 'inception'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(3631, 1.0000001),
 (3629, 0.99565727),
 (3632, 0.9930147),
 (3630, 0.9929747),
 (3628, 0.9911064),
 (3575, 0.9747199),
 (3579, 0.97344255),
 (3576, 0.9720203),
 (3578, 0.9709173),
 (3583, 0.9705367),
 (3573, 0.970334),
 (3572, 0.97006625),
 (3580, 0.96991056),
 (3574, 0.96971935),
 (3577, 0.969381)]

In [27]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['inception',
 'mission impossible',
 'gravity',
 'get out',
 'minority report',
 "Brother's Kiss, A (1997)",
 'Last of the High Kings, The (a.k.a. Summer Fling) (1996)',
 'War at Home, The (1996)',
 'Number Seventeen (1932)',
 'Neon Bible, The (1995)',
 'Male and Female (1919)',
 "I Don't Want to Talk About It (De eso no se habla) (1993)",
 "Another Man's Poison (1952)",
 'Century (1993)',
 'Daens (1992)']

In [28]:
# 비슷한 영화 이름 검색 함수
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [29]:
get_similar_movie('get out')

['get out',
 'gravity',
 'mission impossible',
 'minority report',
 'inception',
 "Brother's Kiss, A (1997)",
 'Last of the High Kings, The (a.k.a. Summer Fling) (1996)',
 'War at Home, The (1996)',
 'Neon Bible, The (1995)',
 "I Don't Want to Talk About It (De eso no se habla) (1993)"]

In [30]:
user = user_to_idx['mino']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(126, 0.0055013974),
 (60, 0.0053397818),
 (2229, 0.0048343064),
 (232, 0.0045900573),
 (107, 0.004383673),
 (41, 0.004264715),
 (258, 0.004214205),
 (2381, 0.004163033),
 (2593, 0.004119696),
 (689, 0.0040744613),
 (707, 0.004047567),
 (3309, 0.0040117223),
 (1830, 0.0039538876),
 (2628, 0.00392361),
 (161, 0.0038668439),
 (2894, 0.0038426183),
 (2376, 0.003816321),
 (2609, 0.0037773724),
 (1695, 0.0037678487),
 (1154, 0.0037504584)]

In [31]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Shakespeare in Love (1998)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Thieves (Voleurs, Les) (1996)',
 'Beautiful Thing (1996)',
 'Jurassic Park (1993)',
 'Rain Man (1988)',
 'Wild Reeds (1994)',
 'Open Your Eyes (Abre los ojos) (1997)',
 'My Favorite Season (1993)',
 'Anne Frank Remembered (1995)',
 'Sting, The (1973)',
 'Hollow Reed (1996)',
 'Return of the Fly (1959)',
 'Get Real (1998)',
 'Graduate, The (1967)',
 'When Night Is Falling (1995)',
 'Xiu Xiu: The Sent-Down Girl (Tian yu) (1998)',
 'Great White Hype, The (1996)',
 'Messenger: The Story of Joan of Arc, The (1999)',
 'Blue Sky (1994)']

# 회고

요즘 여러 플랫폼에서 기본적으로 사용하고 있는 추천 시스템을 사용해볼 수 있어서 좋았다.

또 csr matrix라는 처음 접해봤는데,

이런 방법으로 낭비되는 메모리를 줄일 수 있다는게 신기했다.